## 作業
礙於不是所有同學都有 GPU ，這邊的範例使用的是簡化版本的 ResNet，確保所有同學都能夠順利訓練!


最後一天的作業請閱讀這篇非常詳盡的[文章](https://blog.gtwang.org/programming/keras-resnet-50-pre-trained-model-build-dogs-cats-image-classification-system/)，基本上已經涵蓋了所有訓練　CNN 常用的技巧，請使用所有學過的訓練技巧，盡可能地提高 Cifar-10 的 test data 準確率，截圖你最佳的結果並上傳來完成最後一次的作業吧!

另外這些技巧在 Kaggle 上也會被許多人使用，更有人會開發一些新的技巧，例如使把預訓練在 ImageNet 上的模型當成 feature extractor 後，再拿擷取出的特徵重新訓練新的模型，這些技巧再進階的課程我們會在提到，有興趣的同學也可以[參考](https://www.kaggle.com/insaff/img-feature-extraction-with-pretrained-resnet)

In [14]:
import warnings
warnings.filterwarnings('ignore')
import os

In [15]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [23]:
from keras.datasets import cifar10
from resnet_builder import resnet # 這是從 resnet_builder.py 中直接 import 撰寫好的 resnet 函數
from keras.models import Model
from keras.optimizers import Adam
from keras.utils import to_categorical
import numpy as np

In [17]:
# 讀取資料集並作前處理
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train = x_train / 255.
x_test = x_test / 255.
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [18]:
# 學習率動態調整。當跑到第幾個 epcoh 時，根據設定修改學習率。這邊的數值都是參考原 paper
def lr_schedule(epoch):
    """Learning Rate Schedule
    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.
    # Arguments
        epoch (int): The number of epochs
    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

In [20]:
from keras.callbacks import ModelCheckpoint, LearningRateScheduler

# 使用動態調整學習率
lr_scheduler = LearningRateScheduler(lr_schedule)

In [24]:
from keras.callbacks import ReduceLROnPlateau
# 使用自動降低學習率 (當 validation loss 連續 5 次沒有下降時，自動降低學習率)
lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

In [25]:
# 設定 callbacks
callbacks = [lr_reducer, lr_scheduler]

In [26]:
# 建立 ResNet 模型
model = resnet(input_shape=(32,32,3)) 
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_32 (Conv2D)              (None, 32, 32, 16)   448         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_29 (BatchNo (None, 32, 32, 16)   64          conv2d_32[0][0]                  
__________________________________________________________________________________________________
activation_29 (Activation)      (None, 32, 32, 16)   0           batch_normalization_29[0][0]     
____________________________________________________________________________________________

In [27]:
from keras.preprocessing.image import ImageDataGenerator

In [28]:
batch_size = 64 # batch 的大小，如果出現 OOM error，請降低這個值
num_classes = 10 # 類別的數量，Cifar 10 共有 10 個類別
epochs = 30 # 訓練整個資料集共 30個循環
data_augmentation = True
# 資料標準化的方式，此處使用減去所有影像的平均值
subtract_pixel_mean = True



model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

augment_generator = ImageDataGenerator(rotation_range=10, width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)

# 將資料送進 ImageDataGenrator 中做增強
augment_generator.fit(x_train)

In [29]:
# run Training
history = model.fit_generator(augment_generator.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch=int(len(x_train)/batch_size), # 一個 epochs 要執行幾次 update，通常是資料量除以 batch size
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test),
                    callbacks = callbacks,
                    workers=4)
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/30
Learning rate:  0.001
781/781 [==============================] - 707s 905ms/step - loss: 1.8899 - accuracy: 0.4889 - val_loss: 2.5343 - val_accuracy: 0.3966
Epoch 2/30
Learning rate:  0.001
781/781 [==============================] - 720s 921ms/step - loss: 1.4246 - accuracy: 0.6216 - val_loss: 1.7473 - val_accuracy: 0.5361
Epoch 3/30
Learning rate:  0.001
781/781 [==============================] - 719s 920ms/step - loss: 1.2266 - accuracy: 0.6802 - val_loss: 1.5736 - val_accuracy: 0.5653
Epoch 4/30
Learning rate:  0.001
781/781 [==============================] - 710s 910ms/step - loss: 1.1090 - accuracy: 0.7151 - val_loss: 1.4810 - val_accuracy: 0.5939
Epoch 5/30
Learning rate:  0.001
781/781 [==============================] - 698s 893ms/step - loss: 1.0227 - accuracy: 0.7431 - val_loss: 1.2552 - val_accuracy: 0.6847
Epoch 6/30
Learning rate:  0.001
781/781 [==============================] - 821s 1s/step - loss: 0.9598 - accuracy: 0.7622 - val_loss: 1.0945 - val_accuracy: 0.